# Arcanoid

**Imports**

In [1]:
import cv2
import random
import numpy as np

**Class Ball**

In [2]:
class Ball:
    # Constructor
    def __init__(self, frame, speed = 4):
        # User points
        self.points = 0
        
        # Radius
        self.radius = 8
        
        # Coordinates
        self.x = frame.shape[1]//2
        self.y = int(frame.shape[0]*0.7) - self.radius + 1
        
        # Speeds on axis
        self.move_x = speed
        self.move_y = -speed
        
        # Whether ball on the board or not
        self.out = False
    
    
    # Printing ball on its current position
    def show(self, frame):
        cv2.circle(frame, (self.x, self.y), self.radius, (0,0,255), -1)
    
    
    # Whether ball hit the wall | platform or not 
    def check_boundaries(self, frame, platform):
        next_x = self.x + self.move_x + self.move_x/abs(self.move_x)*self.radius
        next_y = self.y + self.move_y + self.move_y/abs(self.move_y)*self.radius
        
        
        # Check boundaries
        if (0 >= next_x or next_x >= frame.shape[1]):
            self.move_x = -self.move_x
        elif next_y <= 0:
            self.move_y = -self.move_y
        elif next_y >= frame.shape[0]:
            self.out = True
        
        
        # Check platform
        if platform.x1 <= next_x <= platform.x2:
            if platform.y1 <= next_y <= platform.y2:
                dx = min(abs(platform.x1 - next_x), abs(platform.x2 - next_x))
                dy = min(abs(platform.y1 - next_y), abs(platform.y2 - next_y))
                
                if (dx < dy):
                    self.move_x = -self.move_x
                elif (dx > dy):
                    self.move_y = -self.move_y
                else:
                    self.move_x = -self.move_x
                    self.move_y = -self.move_y
                    
        
    
    # Whether ball hit the target brick
    def check_bitting(self, blocks):
        block_to_delete = Block(0,0)
        min_dist = 10**9
        
        next_x = self.x + self.move_x + self.move_x/abs(self.move_x)*self.radius
        next_y = self.y + self.move_y + self.move_y/abs(self.move_y)*self.radius
        
        for block in blocks:
            if block.x1 - 8 <= next_x <= block.x2 + 8:
                if block.y1 - 8 <= next_y <= block.y2 + 8:
                    center_x = block.x1 + 10
                    center_y = block.y1 + 10
                    
                    dist = ((self.x-center_x)**2 + (self.y - center_y)**2)**0.5
                    
                    if dist < min_dist:
                        dist = min_dist
                        block_to_delete = block
                    
        if not ((block_to_delete.x1 == block_to_delete.y1 == 0) and\
        (block_to_delete.x2 == block_to_delete.y2)):
            center_x = block_to_delete.x1 + 10
            center_y = block_to_delete.y1 + 10

            if (self.x-center_x) > (self.y - center_y):
                self.move_x = -self.move_x
            elif (self.x-center_x) < (self.y - center_y):
                self.move_y = -self.move_y
            else:
                self.move_x = -self.move_x
                self.move_y = -self.move_y

            block_to_delete.destruct()
            self.points += 10

        
    # Moving ball to its next point
    def move(self, frame, platform, blocks):
        # Check rebound
        self.check_boundaries(frame, platform)
        
        # Check hitting the target
        self.check_bitting(blocks)
        
        # Move ball
        self.x += self.move_x
        self.y += self.move_y

In [3]:
class Platform:
    
    # Constructor
    def __init__(self, frame):
        self.step = 15
        self.width = 80
        self.bound = 5
        
        self.x1 = frame.shape[1]//2 - self.width//2
        self.x2 = frame.shape[1]//2 + self.width//2
        
        self.y1 = int(frame.shape[0]*0.9)
        self.y2 = self.y1 + 20
        
    
    # Printing ball on its current position
    def show(self, frame):
        cv2.rectangle(frame, (self.x1, self.y1), (self.x2, self.y2), (0, 255, 0), -1)
        
    
    def check_boundaries(self, frame):
        
        if 0 >= self.x1:
            self.x1 += self.bound
            self.x2 += self.bound
            return False
        elif frame.shape[1] <= self.x2:
            self.x1 -= self.bound
            self.x2 -= self.bound
            return False
        return True
        

    # Moving ball to its next point (by key)
    def move(self, frame, key):
        if self.check_boundaries(frame):
            if key == 83:
                self.x1 += self.step
                self.x2 += self.step
            elif key == 81:
                self.x1 -= self.step
                self.x2 -= self.step
    
    
    # Moving ball to its next point (by visual)
    def move_with_contour(self, frame, approx):
        if self.check_boundaries(frame) and (approx[0][0][0] != approx[0][0][1]):
            self.x1 = approx[0][0][0]
            self.x2 = self.x1 + self.width

**Class Block**

In [4]:
class Block:
    # Constructor
    def __init__(self, x, y):
        self.x1 = x
        self.x2 = self.x1 + 20
        self.y1 = y
        self.y2 = self.y1 + 20
    
    # Printing ball on its current position
    def show(self, frame):
        if not (self.x1 == self.x2 == self.y1 == self.y2 == 0):
            cv2.rectangle(frame, (self.x1+1, self.y1+1), (self.x2-1, self.y2-1), (0, 255, 255), -1)
    
    # Delete block
    def destruct(self):
        self.x1, self.x2 = 0, 0
        self.y1, self.y2 = 0, 0

**Function that'll generate blocks**

In [5]:
def generate_blocks(frame):
    size = 20
    blocks = []
    x = [i for i in range(10, frame.shape[1]-size, size)]
    y = [[j for j in range(10, frame.shape[0]//3, size)] for i in range(10, frame.shape[1]-size, size)]
    
    for i in range(int(len(x)*len(y[0])*0.9)):
        # Getting coordinates
        x_need = random.choice(x)
        y_need = random.choice(y[x.index(x_need)])
        
        # Removing them
        y[x.index(x_need)].remove(y_need)
        if (len(y[x.index(x_need)]) == 0):
            del y[x.index(x_need)]
            x.remove(x_need)

        blocks.append(Block(x_need, y_need))
    
    return blocks

**Main function**

In [14]:
font = cv2.FONT_HERSHEY_COMPLEX

l_h = 0
l_s = 0
l_v = 80
u_h = 255
u_s = 255
u_v = 255

cap = cv2.VideoCapture(0)
played = False
while True:
    # Intro
    key = cv2.waitKey(1)
    _, frame = cap.read()
#     frame = cv2.flip(frame, 1)
    
    # Introduction text or end of game text
    if (not played):
        cv2.putText(frame, "HELLO !!!", (240, 240), font, 1, (0, 0, 255))
        cv2.putText(frame, "PRESS 'SPACE' TO START", (100, 280), font, 1, (0, 0, 255))
    else:
        cv2.putText(frame, "GAME OVER", (240, 240), font, 1, (0, 0, 255))
        cv2.putText(frame, "SCORE : " + str(ball.points), (240, 280), font, 1, (0, 0, 255))
    
    cv2.imshow("Arcanoid", frame)
    
    
    # If SPACE pressed (32 - code of SPACE)
    if key == 32:
        played = True
        
        ball = Ball(frame, 3)
        platform = Platform(frame)
        blocks = generate_blocks(frame)
        
        while True:
            _, frame = cap.read()
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            
            # Showing figures
            cv2.putText(frame, "SCORE : " + str(ball.points), (30, 30), font, 1, (255, 0, 0))
            ball.show(frame)
            platform.show(frame)
            for block in blocks:
                block.show(frame)


            #Moving objects
            #-----------------------------------------------------------------------------------------------

            lower_red = np.array([l_h, l_s, l_v])
            upper_red = np.array([u_h, u_s, u_v])

            mask = cv2.inRange(hsv, lower_red, upper_red)
            kernel = np.ones((5, 5), np.uint8)
            mask = cv2.erode(mask, kernel)

            # Contours detection
            contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            font = cv2.FONT_HERSHEY_COMPLEX


            for cnt in contours:
                area = cv2.contourArea(cnt)
                approx = cv2.approxPolyDP(cnt, 0.02*cv2.arcLength(cnt, True), True)
                x = approx.ravel()[0]
                y = approx.ravel()[1]

                if  8000 > area > 400:
                    if len(approx) == 4:
                        platform.move_with_contour(frame, approx)


            cv2.putText(frame, "SCORE : " + str(ball.points), (30, 30), font, 1, (255, 255, 255))
            cv2.imshow("Arcanoid", frame)
            cv2.imshow("Mask", mask)
            #----------------------------------------------------------------------------------------------------

            
            # If ESC was pressed or ball gone out -> GAME OVER
            key = cv2.waitKey(1)
            if key == 27 or ball.out:
                ball.out = True
                break

            ball.move(frame, platform, blocks)
            platform.move(frame, key)
            
            for block in blocks:
                if block.x1 == block.x1 == block.x2 == block.x2 == block.y1 == block.y1 == block.y2 == block.y2 == 0:
                    blocks.remove(block)
    
    elif key == 27:
        break
    

cap.release()
cv2.destroyAllWindows()